In [0]:
select COUNT(*) from telco_external.landing_telco_customer_churn_data limit 10;

In [0]:
CREATE SCHEMA telco_internal ;

In [0]:
CREATE TABLE telco_internal.staging_telco_customer_churn_data (CustomerID, Count, Country, State, City, Zip_Code,
    Latitude, Longitude, Gender, SeniorCitizen, Partner, Dependents,
    Tenure, PhoneService, MultipleLines, InternetService, OnlineSecurity,
    OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies,
    Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges,
    Churn_Label, Churn_Value, Churn_Score, CLTV, Churn_Reason) AS
SELECT
    DISTINCT CustomerID, Count, Country, State, City, "zip code",
    Latitude, Longitude, Gender, "senior citizen", Partner, Dependents,
    "tenure months", "phone service", "multiple lines", "internet service", "online security",
    "online backup", "device protection", "tech support", "streaming tv", "streaming movies",
    Contract, "paperless billing", "payment method", "monthly charges", "total charges",
    "churn label", "churn value", "churn score", "cltv", "churn reason" , "record_modified"
FROM telco_external.landing_telco_customer_churn_data;

In [0]:
DROP TABLE IF EXISTS telco_internal.dim_location ;
CREATE TABLE telco_internal.dim_location (
    location_key INT IDENTITY(1,1) PRIMARY KEY,
    Country VARCHAR(255),
    State VARCHAR(255),
    City VARCHAR(255),
    Zip_Code INT,
    Latitude FLOAT,
    Longitude FLOAT
);


INSERT INTO telco_internal.dim_location (Country, State, City, Zip_Code, Latitude, Longitude)
SELECT DISTINCT Country, State, City, Zip_Code, Latitude, Longitude
FROM telco_internal.staging_telco_customer_churn_data;

In [0]:
CREATE TABLE IF NOT EXISTS telco_internal.dim_service (
    service_key INT IDENTITY(1,1) PRIMARY KEY,
    CustomerID VARCHAR(255),
    PhoneService VARCHAR(50),
    MultipleLines VARCHAR(50),
    InternetService VARCHAR(50),
    OnlineSecurity VARCHAR(50),
    OnlineBackup VARCHAR(50),
    DeviceProtection VARCHAR(50),
    TechSupport VARCHAR(50),
    StreamingTV VARCHAR(50),
    StreamingMovies VARCHAR(50)
);
INSERT INTO dim_service (CustomerID, PhoneService, MultipleLines, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies)
SELECT DISTINCT CustomerID, PhoneService, MultipleLines, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies
FROM telco_internal.staging_telco_customer_churn_data;

In [0]:
DROP TABLE IF EXISTS telco_internal.dim_customer ;

CREATE TABLE IF NOT EXISTS telco_internal.dim_customer (
    customer_key INT IDENTITY(1,1) PRIMARY KEY,
    CustomerID VARCHAR(255),
    Gender VARCHAR(50),
    SeniorCitizen VARCHAR(50),
    Partner VARCHAR(50),
    Dependents VARCHAR(50),
    EffectiveDate DATE DEFAULT CURRENT_DATE,
    EndDate DATE DEFAULT '9999-12-31',
    IsCurrent BOOLEAN DEFAULT TRUE
);

In [0]:
-- Add a new version of a customer record if there's a change
INSERT INTO telco_internal.dim_customer (CustomerID, Gender, SeniorCitizen, Partner, Dependents, EffectiveDate, IsCurrent)
SELECT
    sc.CustomerID,
    sc.Gender,
    sc.SeniorCitizen,
    sc.Partner,
    sc.Dependents,
    CURRENT_DATE,
    TRUE
FROM telco_internal.staging_telco_customer_churn_data sc
LEFT JOIN telco_internal.dim_customer dc ON sc.CustomerID = dc.CustomerID
WHERE dc.IsCurrent = TRUE
  AND (sc.Gender <> dc.Gender OR sc.SeniorCitizen <> dc.SeniorCitizen OR sc.Partner <> dc.Partner OR sc.Dependents <> dc.Dependents);

In [0]:
-- Mark the old version as not current
UPDATE telco_internal.dim_customer
SET EndDate = CURRENT_DATE,
    IsCurrent = FALSE
WHERE CustomerID IN (
    SELECT sc.CustomerID
    FROM telco_internal.staging_telco_customer_churn_data sc
    LEFT JOIN telco_internal.dim_customer dc ON sc.CustomerID = dc.CustomerID
    WHERE dc.IsCurrent = TRUE
      AND (sc.Gender <> dc.Gender OR sc.SeniorCitizen <> dc.SeniorCitizen OR sc.Partner <> dc.Partner OR sc.Dependents <> dc.Dependents)
);

In [0]:
CREATE TABLE telco_internal.staging_fact_churn AS
SELECT
    sc.CustomerID,
    sc.Churn_Label,
    sc.Churn_Value,
    sc.Churn_Score,
    sc.CLTV,
    sc.Churn_Reason,
    dc.customer_key,
    dl.location_key,
    ds.service_key
FROM telco_internal.staging_telco_customer_churn_data sc
JOIN telco_internal.dim_customer dc ON sc.CustomerID = dc.CustomerID AND dc.IsCurrent = TRUE
JOIN telco_internal.dim_location dl ON sc.Zip_Code = dl.Zip_Code
JOIN telco_internal.dim_service ds ON sc.CustomerID = ds.CustomerID

In [0]:
CREATE TABLE telco_internal.fact_churn (
    churn_key INT IDENTITY(1,1) PRIMARY KEY,
    customer_key INT,
    location_key INT,
    service_key INT,
    billing_key INT,
    Churn_Label VARCHAR(50),
    Churn_Value INT,
    Churn_Score INT,
    CLTV INT,
    Churn_Reason VARCHAR(255),
    FOREIGN KEY (customer_key) REFERENCES telco_internal.dim_customer(customer_key),
    FOREIGN KEY (location_key) REFERENCES telco_internal.dim_location(location_key),
    FOREIGN KEY (service_key) REFERENCES telco_internal.dim_service(service_key)
);

In [0]:
INSERT INTO telco_internal.fact_churn (customer_key, location_key, service_key, Churn_Label, Churn_Value, Churn_Score, CLTV, Churn_Reason)
SELECT
    customer_key,
    location_key,dim_billing
    service_key,
    Churn_Label,
    Churn_Value,
    Churn_Score,
    CLTV,
    Churn_Reason
FROM telco_internal.staging_fact_churn;